In [ ]:
import sys
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib

Example uses Pacers team ID to find games

In [ ]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

pacers = [team for team in nba_teams if team['abbreviation'] == 'IND'][0]
pacers_id = pacers['id']
print(f'pacers_id: {pacers_id}')

In [4]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=pacers_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

# games_dict = gamefinder.get_normalized_dict()
# games = games_dict['LeagueGameFinderResults']
# game = games[0]
# game_id = game['GAME_ID']
# game_matchup = game['MATCHUP']

# print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game_ids = [(game["GAME_ID"], game["GAME_DATE"]) for game in games]
# game_ids

In [ ]:
len(game_ids)

In [ ]:
from nba_api.stats.endpoints import playbyplayv3

df_list = [playbyplayv3.PlayByPlayV3(id[0]).get_data_frames()[0] for id in game_ids[-10:]]

In [ ]:
import pandas as pd

df = pd.concat(df_list)
df = df[['gameId', 'actionNumber', 'clock', 'period', 'teamId', 'teamTricode', 'personId', 
         'playerNameI', 'xLegacy', 'yLegacy', 'shotDistance', 'shotResult', 'isFieldGoal', 
         'scoreHome', 'scoreAway', 'location', 'actionType', 'subType', 'shotValue', 'actionId']]
# df.shape[0]
# df.columns

Two filtering methods explored for getting shot events, apparently they are equivalent

In [ ]:
df1 = df[df["actionType"].isin(["Made Shot", "Missed Shot"])]
# df2 = df[df["shotResult"].isin(['Missed', 'Made'])]

# print(f'Original df len: {df.shape[0]}, actionType filter len: {df1.shape[0]}, shotResult filter len: {df2.shape[0]}')

In [ ]:
# df1.head()

Data exploration in form of a scatter plot. Variable we want to explain is "shotResult" ("Made"/"Missed"). Key variables for classification: "shotDistance", "shotValue".
TODO: player shooting stats, defender distance, blocked shots not included...

In [ ]:
import matplotlib.pyplot as plt

# Create the scatter plot
plt.figure(figsize=(7, 7))  # Set figure size
plt.scatter(df1['xLegacy'], df1['yLegacy'], c='blue', alpha=0.6, edgecolors='black')

# Labels and title
plt.xlabel("X Coordinate (Court)")
plt.ylabel("Y Coordinate (Court)")
plt.title("NBA Shot Locations")

# Display the plot
plt.show()

In [12]:
df1.to_csv("data_v1", index=False)